In [1]:
import sys
sys.path.append('C:\\Users\\Patrick\\Documents\\GitHub\\ICDeepLearning\\Scripts')
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, utils, datasets, models
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import numpy as np

from utils import path
from DataVis import createInputFigure
from NNUtils import *
from NNCreationAux import *

from sklearn import preprocessing
import os
import time
import copy

***

In [2]:
modelType = 'dense'
modelSize = '161'
data_type = '512'
figSize = '128'
folder = 'GAF_Combinados_difference'

model_name = modelType+modelSize

In [ ]:
dataloader, dataloaders, dataset_sizes, class_names = getData(data_type, folder)

In [ ]:
model = model_selection(modelName, numClasses)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum = 0.4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 10, gamma = 0.1)
model, stats = train(model, dataloaders, dataset_sizes, criterion, optimizer, scheduler, num_epochs = 2)

***

In [2]:
data_type = '256'
folder = 'RP_Combinados'
path = 'C:\\Users\\Patrick\\Documents\\GitHub\\ICDeepLearning\\Data\\Imagens\\'+folder+'\\'+data_type
dataset = get_dataset(path, 224)
dataloader, dataloaders, dataset_sizes, class_names = getData(data_type, folder, batch_size = 64)

In [3]:
model = model_selection('resnet50', 5).to(device)
optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum = 0.9)
criterion = nn.CrossEntropyLoss()
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 7, gamma = 0.1)

In [ ]:
model, _, _ = kfold_train(model, dataset, criterion, optimizer, num_epochs = 20, num_folds = 5)

Fold 0
--
Epoch 1/20
-----
Stage: train
Loss: 1.7121 - Accuracy: 0.2283
Stage: test
Loss: 1.5747 - Accuracy: 0.2784
--
Epoch 2/20
-----
Stage: train
Loss: 1.5111 - Accuracy: 0.3436
Stage: test
Loss: 1.3460 - Accuracy: 0.4286
--
Epoch 3/20
-----
Stage: train
Loss: 1.2339 - Accuracy: 0.5050
Stage: test
Loss: 1.0169 - Accuracy: 0.6227
--
Epoch 4/20
-----
Stage: train
Loss: 0.8901 - Accuracy: 0.6676
Stage: test
Loss: 0.5251 - Accuracy: 0.8370
--
Epoch 5/20
-----
Stage: train
Loss: 0.6232 - Accuracy: 0.7729
Stage: test
Loss: 0.3324 - Accuracy: 0.8956
--
Epoch 6/20
-----
Stage: train
Loss: 0.4416 - Accuracy: 0.8446
Stage: test
Loss: 0.3122 - Accuracy: 0.9002
--
Epoch 7/20
-----
Stage: train
Loss: 0.3405 - Accuracy: 0.8901
Stage: test
Loss: 0.1876 - Accuracy: 0.9524
--
Epoch 8/20
-----
Stage: train
Loss: 0.2699 - Accuracy: 0.9119
Stage: test
Loss: 0.2070 - Accuracy: 0.9460
--
Epoch 9/20
-----
Stage: train
Loss: 0.2201 - Accuracy: 0.9233
Stage: test
Loss: 0.1347 - Accuracy: 0.9661
--
Epoch 10/

***

In [5]:
batch_size = 1
image_size = 224
folder = 'Test_GASF'
figPath = 'C:\\Users\\Patrick\\Documents\\GitHub\\ICDeepLearning\\Data\\'+folder+'\\512'
dataloader, dataloaders, dataset_sizes, class_names = getData(data_type, folder, batch_size = 1)

In [6]:
class predictor(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
        
    def predict(self, dataloader):
        outputs = []
        targets = []
        for sample, target in dataloader:
            sample = sample.to(device)
            target = target.to(device)
            o = torch.argmax(F.softmax(self.model(sample), dim = 1)).item()
            outputs.append(o)
            targets.append(target.item())
            
        outputs = np.array(outputs)
        targets = np.array(targets)
        right_pred = 1*(outputs == targets)
        return outputs, right_pred

In [14]:
predic = predictor(res18)
_, outres18 = predic.predict(dataloader)

In [15]:
predic = predictor(res34)
_, outres34 = predic.predict(dataloader)

In [7]:
predic = predictor(model)
_, outmodel = predic.predict(dataloader)

In [8]:
np.count_nonzero(outmodel)/500

0.188

In [19]:
np.count_nonzero(outres18)/500

0.178

In [20]:
np.count_nonzero(outres34)/500

0.202

In [22]:
from scipy import stats
stats.ttest_ind(outres18, outres34, equal_var=True)

Ttest_indResult(statistic=-0.9667862845385763, pvalue=0.33388509993212834)